### Build a model to predict engagement with image labels (text) as predictors. Is this model better than using captions to predict the same? What if you used both image labels and captions to predict engagement? 

In [36]:
import pandas as pd
import numpy as np
gv = pd.read_csv("gv_labels.csv")
insta = pd.read_csv("insta_withid.csv")

### Importing captions(hashtags) and cleaning them to create strings

In [37]:
import ast
insta1 = insta[['id_image','Engagement_score','photo','hashtags']]

# Converting strings to list in hashtags
insta1['hashtags1'] = insta1['hashtags'].apply(lambda x : ast.literal_eval(x))

# Concatenating the list of hashtags to be fed into count vectorizer
insta1['hashtags2'] = insta1['hashtags1'].apply(lambda x: ' '.join(x))

# converting to lower case
insta1['hashtags2'] = insta1['hashtags2'].apply(lambda x: x.lower())

insta1.head()

C:\Users\Akhil\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\Akhil\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Akhil\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pa

,id_image,Engagement_score,photo,hashtags,hashtags1,hashtags2
0,1981399443030738687,-0.635760,1,"['happypills', 'pills', 'pharma', 'bigpharma']","[happypills, pills, pharma, bigpharma]",happypills pills pharma bigpharma
1,1980599326570032905,-0.075393,1,"['1', '2']","[1, 2]",1 2
2,1980506709442276709,-0.381748,1,"['FollowMe', 'Madagascar', 'Enoughness', 'natu...","[FollowMe, Madagascar, Enoughness, nature]",followme madagascar enoughness nature
3,1980429175895768953,1.064558,1,"['snowstorm', 'penguin', 'antarctica']","[snowstorm, penguin, antarctica]",snowstorm penguin antarctica
4,1980332447939109605,0.036183,1,"['whales', 'humpbackwhales', 'parenting', 'pla...","[whales, humpbackwhales, parenting, planetofth...",whales humpbackwhales parenting planetofthewhales


### Prediction using hashtags as features
### Random forest

In [40]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(insta1['hashtags2'])
count_vect.get_feature_names()
X_matrix= X_train_counts.todense()

y = np.array(insta1['Engagement_score'])

# Creating the train and test split
from sklearn.model_selection import train_test_split
X_train_m, X_val_m, y_train_m, y_val_m = train_test_split(X_train_counts, y, test_size=0.3, random_state=1)

# Random Forest
from sklearn.ensemble import RandomForestRegressor
rf_insta = RandomForestRegressor(n_estimators = 500, random_state = 42)
rf_insta.fit(X_train_m, y_train_m)
predictions_rf = rf_insta.predict(X_val_m)

In [46]:
rmse_rf = np.sqrt(np.mean((predictions_rf - y_val_m)**2))
print('RMSE using random forest on instagram captions is:', round(rmse_rf, 2))

RMSE using random forest on instagram captions is: 0.53


### XGBoost

In [47]:
import xgboost as xgb
from xgboost import XGBRegressor

xgb_params = {
    'n_estimators': 700, 
    'max_depth': 5,
    'subsample': 0.9,
    'colsample_bytree': 0.9,
    'objective':'reg:linear',
    'learning_rate': 0.01   
}

model = XGBRegressor(**xgb_params).fit(X_train_m, y_train_m)

preds_xgb = model.predict(X_val_m)


rmse_xgb = np.sqrt(np.mean((preds_xgb - y_val_m)**2))
print('RMSE using XGB on instagram captions is:', round(rmse_xgb, 2))

RMSE using XGB on instagram captions is: 0.52


### Using image labels obtained from Google cloud Vision API

In [31]:
gv.head()

,Unnamed: 0,imageid,labels,scores
0,0,1887675048681946284,"['Reflection', 'Water', 'Blue', 'Waterway', 'S...","[0.9603376388549805, 0.9512578248977661, 0.937..."
1,1,1887750972126714187,"['Sport venue', 'Product', 'Stadium', 'Fan', '...","[0.9421510100364685, 0.9413959383964539, 0.921..."
2,2,1887939394339128382,"['Eyewear', 'Glasses', 'Cool', 'Nail', 'Hand',...","[0.9922423958778381, 0.9812176823616028, 0.893..."
3,3,1888007540646834631,"['Hair', 'Face', 'Hairstyle', 'Forehead', 'Hea...","[0.9909557700157166, 0.9656747579574585, 0.960..."
4,4,1888083309374639933,"['Transport', 'Green', 'Mode of transport', 'P...","[0.9205442667007446, 0.9068462252616882, 0.808..."


In [38]:
import ast
import pandas as pd

# Converting strings to list in hashtags
gv['labels1'] = gv['labels'].apply(lambda x : ast.literal_eval(x))

# # Concatenating the list of hashtags to be fed into count vectorizer
gv['labels2'] = gv['labels1'].apply(lambda x: ' '.join(x))

# converting to lower case
gv['labels2'] = gv['labels2'].apply(lambda x: x.lower())

# # merging to get engagement scores
gv1 = pd.merge(insta1,gv,how = 'inner',left_on ='id_image',right_on = 'imageid')

In [56]:
gv1.head()

,id_image,Engagement_score,photo,hashtags,hashtags1,hashtags2,Unnamed: 0,imageid,labels,scores,labels1,labels2,final_vector
0,1981399443030738687,-0.635760,1,"['happypills', 'pills', 'pharma', 'bigpharma']","[happypills, pills, pharma, bigpharma]",happypills pills pharma bigpharma,1090,1981399443030738687,"['Leisure', 'Party', 'Drink', 'Vacation', 'Sit...","[0.6388171315193176, 0.631462812423706, 0.6256...","[Leisure, Party, Drink, Vacation, Sitting, Fur...",leisure party drink vacation sitting furniture...,happypills pills pharma bigpharmaleisure party...
1,1980599326570032905,-0.075393,1,"['1', '2']","[1, 2]",1 2,1089,1980599326570032905,"['Mountainous landforms', 'Mountain', 'Mountai...","[0.9923034310340881, 0.987432062625885, 0.9084...","[Mountainous landforms, Mountain, Mountain ran...",mountainous landforms mountain mountain range ...,1 2mountainous landforms mountain mountain ran...
2,1980506709442276709,-0.381748,1,"['FollowMe', 'Madagascar', 'Enoughness', 'natu...","[FollowMe, Madagascar, Enoughness, nature]",followme madagascar enoughness nature,1088,1980506709442276709,"['Tree', 'Adaptation', 'Working animal', 'Chil...","[0.9000945091247559, 0.840880811214447, 0.7956...","[Tree, Adaptation, Working animal, Child, Gras...",tree adaptation working animal child grass pla...,followme madagascar enoughness naturetree adap...
3,1980429175895768953,1.064558,1,"['snowstorm', 'penguin', 'antarctica']","[snowstorm, penguin, antarctica]",snowstorm penguin antarctica,1087,1980429175895768953,"['Bird', 'Penguin', 'Vertebrate', 'Flightless ...","[0.9960000514984131, 0.9883484244346619, 0.985...","[Bird, Penguin, Vertebrate, Flightless bird, B...",bird penguin vertebrate flightless bird beak a...,snowstorm penguin antarcticabird penguin verte...
4,1980332447939109605,0.036183,1,"['whales', 'humpbackwhales', 'parenting', 'pla...","[whales, humpbackwhales, parenting, planetofth...",whales humpbackwhales parenting planetofthewhales,1086,1980332447939109605,"['Marine mammal', 'Marine biology', 'Cetacea',...","[0.945663571357727, 0.9357404112815857, 0.9342...","[Marine mammal, Marine biology, Cetacea, Under...",marine mammal marine biology cetacea underwate...,whales humpbackwhales parenting planetofthewha...


### Predictions
### Random forest

In [49]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(gv1['labels2'])
count_vect.get_feature_names()
X_matrix= X_train_counts.todense()

y = np.array(gv1['Engagement_score'])

# Creating the train and test split
from sklearn.model_selection import train_test_split
X_train_m, X_val_m, y_train_m, y_val_m = train_test_split(X_train_counts, y, test_size=0.3, random_state=1)

# Random Forest
from sklearn.ensemble import RandomForestRegressor
rf_insta = RandomForestRegressor(n_estimators = 700, random_state = 42)
rf_insta.fit(X_train_m, y_train_m)
predictions_rf = rf_insta.predict(X_val_m)

In [80]:
word_df = pd.DataFrame(X_matrix,columns = count_vect.vocabulary_)

In [50]:
rmse_rf = np.sqrt(np.mean((predictions_rf - y_val_m)**2))
print('RMSE using random forest on instagram captions is:', round(rmse_rf, 2))

RMSE using random forest on instagram captions is: 1.07


### XGboost

In [51]:
import xgboost as xgb
from xgboost import XGBRegressor

xgb_params = {
    'n_estimators': 700, 
    'max_depth': 5,
    'subsample': 0.9,
    'colsample_bytree': 0.9,
    'objective':'reg:linear',
    'learning_rate': 0.01   
}

model = XGBRegressor(**xgb_params).fit(X_train_m, y_train_m)

preds_xgb = model.predict(X_val_m)


rmse_xgb = np.sqrt(np.mean((preds_xgb - y_val_m)**2))
print('RMSE using XGB on instagram captions is:', round(rmse_xgb, 2))

RMSE using XGB on instagram captions is: 1.06


## Using both captions(hashtags) and image labels

In [52]:
gv1['final_vector'] = gv1.apply(lambda row : row['hashtags2'] + row['labels2'],axis = 1)

### Random forest

In [53]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(gv1['final_vector'])
count_vect.get_feature_names()
X_matrix= X_train_counts.todense()

y = np.array(gv1['Engagement_score'])

# Creating the train and test split
from sklearn.model_selection import train_test_split
X_train_m, X_val_m, y_train_m, y_val_m = train_test_split(X_train_counts, y, test_size=0.3, random_state=1)

# Random Forest
from sklearn.ensemble import RandomForestRegressor
rf_insta = RandomForestRegressor(n_estimators = 700, random_state = 42)
rf_insta.fit(X_train_m, y_train_m)
predictions_rf = rf_insta.predict(X_val_m)

In [54]:
rmse_rf = np.sqrt(np.mean((predictions_rf - y_val_m)**2))
print('RMSE using random forest on instagram captions is:', round(rmse_rf, 2))

RMSE using random forest on instagram captions is: 1.09


### XGBoost

In [55]:
import xgboost as xgb
from xgboost import XGBRegressor

xgb_params = {
    'n_estimators': 700, 
    'max_depth': 5,
    'subsample': 0.9,
    'colsample_bytree': 0.9,
    'objective':'reg:linear',
    'learning_rate': 0.01   
}

model = XGBRegressor(**xgb_params).fit(X_train_m, y_train_m)

preds_xgb = model.predict(X_val_m)


rmse_xgb = np.sqrt(np.mean((preds_xgb - y_val_m)**2))
print('RMSE using XGB on instagram captions is:', round(rmse_xgb, 2))

RMSE using XGB on instagram captions is: 1.06


### Conclusion   
In our case, we can see that hashtags contribute to a smaller rmse in comparison to using just image labels or a combination of hashtags and labels.